In [1]:
## Import libraries

from platform import python_version
print(python_version())
import torch
print(torch.__version__)
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.utils.data as utils
import torch.nn as nn
import torch.nn.functional as F

from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
import h5py as h5

import time
import seaborn as sns
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from   matplotlib import pylab
sns.set_style("whitegrid")
np.random.seed(697)

from sklearn            import preprocessing
from sklearn            import metrics
from sklearn.metrics    import confusion_matrix, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.datasets   import make_classification
from sklearn.neighbors  import KNeighborsClassifier
from sklearn.ensemble   import RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

3.7.3
1.2.0


In [2]:
DATA_DIR = ''
#if torch.cuda.is_available():   
#    torch.set_default_tensor_type('torch.cuda.FloatTensor')
#def to_numpy(tensor):
#    '''converts a GPU tensor to a numpy array'''
#    return tensor.cpu().detach().numpy()

In [3]:
f = h5.File('./el_data.h5', 'r') ## gives group
print('\033[1;35m Key name:\033[1;m', list(f.keys()))

 Key name: ['correctedAverageMu', 'em_barrel_Lr0', 'em_barrel_Lr1', 'em_barrel_Lr1_fine', 'em_barrel_Lr2', 'em_barrel_Lr3', 'eventNumber', 'mcChannelNumber', 'p_ECIDSResult', 'p_Eratio', 'p_LHLoose', 'p_LHMedium', 'p_LHTight', 'p_LHValue', 'p_Reta', 'p_Rhad', 'p_Rphi', 'p_TRTPID', 'p_TruthOrigin', 'p_TruthType', 'p_charge', 'p_chi2', 'p_d0', 'p_d0Sig', 'p_dPOverP', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_e', 'p_et_calo', 'p_eta', 'p_f1', 'p_f3', 'p_firstEgMotherTruthOrigin', 'p_firstEgMotherTruthType', 'p_iffTruth', 'p_mean_charge', 'p_mean_chi2', 'p_mean_d0', 'p_mean_deta', 'p_mean_dphi', 'p_mean_efrac', 'p_mean_ndof', 'p_mean_pixhits', 'p_mean_scthits', 'p_mean_sigmad0', 'p_mean_trthits', 'p_mean_vertex', 'p_mean_z0', 'p_nTracks', 'p_ndof', 'p_numberOfSCTHits', 'p_phi', 'p_pt_track', 'p_qd0Sig', 'p_sct_weight_charge', 'p_sigmad0', 'p_tracks', 'p_truth_eta', 'p_truth_phi', 'p_truth_pt', 'p_weta2', 'tile_barrel_Lr1', 'tile_barrel_Lr2', 'tile_barrel_Lr3', 'tracks', 'true_m']


In [4]:
file_name = h5.File('./el_data.h5', 'r') ## gives group
keys = list(file_name.keys())
print(keys)
#print(list(file_name[keys[0]]))
#print(list(file_name[keys[1]]))
#print(list(file_name[list(file_name.keys())[0]])) ## or
file_name.close()

['correctedAverageMu', 'em_barrel_Lr0', 'em_barrel_Lr1', 'em_barrel_Lr1_fine', 'em_barrel_Lr2', 'em_barrel_Lr3', 'eventNumber', 'mcChannelNumber', 'p_ECIDSResult', 'p_Eratio', 'p_LHLoose', 'p_LHMedium', 'p_LHTight', 'p_LHValue', 'p_Reta', 'p_Rhad', 'p_Rphi', 'p_TRTPID', 'p_TruthOrigin', 'p_TruthType', 'p_charge', 'p_chi2', 'p_d0', 'p_d0Sig', 'p_dPOverP', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_e', 'p_et_calo', 'p_eta', 'p_f1', 'p_f3', 'p_firstEgMotherTruthOrigin', 'p_firstEgMotherTruthType', 'p_iffTruth', 'p_mean_charge', 'p_mean_chi2', 'p_mean_d0', 'p_mean_deta', 'p_mean_dphi', 'p_mean_efrac', 'p_mean_ndof', 'p_mean_pixhits', 'p_mean_scthits', 'p_mean_sigmad0', 'p_mean_trthits', 'p_mean_vertex', 'p_mean_z0', 'p_nTracks', 'p_ndof', 'p_numberOfSCTHits', 'p_phi', 'p_pt_track', 'p_qd0Sig', 'p_sct_weight_charge', 'p_sigmad0', 'p_tracks', 'p_truth_eta', 'p_truth_phi', 'p_truth_pt', 'p_weta2', 'tile_barrel_Lr1', 'tile_barrel_Lr2', 'tile_barrel_Lr3', 'tracks', 'true_m']


In [7]:
def make_sample(data_file, batch_size, all_features, images, upscale=False, denormalize=False, index=0):
    data = h5.File(data_file, 'r')
    idx_1, idx_2 = index*batch_size, (index+1)*batch_size
    sample_dict  = dict([key, data[key][idx_1:idx_2]] for key in all_features)
    if images != [] and denormalize:
        energy = sample_dict['p_e']
        for key in images: sample_dict[key] = sample_dict[key] * energy[:, np.newaxis, np.newaxis]
        sample_dict['tracks'][:,:,0] = sample_dict['tracks'][:,:,0] * energy[:, np.newaxis]
    if images != [] and upscale:
        for i in images: sample_dict[i] = resize_images(np.float32(sample_dict[i]), target_shape=(56,11))
    return sample_dict

In [8]:
images    = ['em_barrel_Lr0',  'em_barrel_Lr1', 'em_barrel_Lr2', 'em_barrel_Lr3',
             'tile_barrel_Lr1', 'tile_barrel_Lr2', 'tile_barrel_Lr3']
#images    = ['em_barrel_Lr1_fine']
tracks    = ['tracks' ]
scalars   = ['p_Eratio', 'p_Reta', 'p_Rhad', 'p_Rphi', 'p_TRTPID', 'p_d0', 'p_d0Sig', 'p_dPOverP',
             'p_deltaPhiRescaled2', 'p_deltaEta1', 'p_f1', 'p_f3', 'p_numberOfSCTHits', 'p_weta2']
others    = ['p_TruthType', 'p_iffTruth', 'p_LHTight', 'p_LHMedium', 'p_LHLoose', 'p_e']
#train_features = {'images':images, 'tracks':tracks, 'scalars':scalars}
train_features = {'images':images, 'tracks':[], 'scalars':[]}
all_features   = np.sum(list(train_features.values())) + others
if train_features['images'] == []: args.n_type = 'FCN'

In [9]:
torch.cuda.get_device_properties(device=None).total_memory

12790923264

In [10]:
data_file = './el_data.h5'

In [11]:
# n_e = 1000000
n_e = 19920
xtrain_data = make_sample(data_file, n_e, all_features, train_features['images'], upscale=False)

In [12]:
train_data = [ np.expand_dims(np.float32(xtrain_data[key]),axis = 1)  for key in np.sum(list(train_features.values()))]
#train_data = [np.float32(train_data[key])[:,np.newaxis,:,:]  for key in np.sum(list(train_features.values()))]
#train_data = [np.float32(train_data[key]) for key in np.sum(list(train_features.values()))]

In [13]:
for element in train_data:
    print(element.shape)

(19920, 1, 7, 11)
(19920, 1, 7, 11)
(19920, 1, 7, 11)
(19920, 1, 7, 11)
(19920, 1, 7, 11)
(19920, 1, 7, 11)
(19920, 1, 7, 11)


In [14]:
x = np.concatenate(train_data, axis=1)
print(x.shape)

(19920, 7, 7, 11)


In [15]:
def make_labels(data, n_classes):
    if   n_classes == 2:
        labels = np.where(np.logical_or(data['p_TruthType']==2, data['p_TruthType']==4), 0, 1)
    elif n_classes == 5:
        truth  = data['p_iffTruth']
        labels = np.where(truth==2, 0, 4     )
        labels = np.where(truth==3, 1, labels)
        labels = np.where(np.logical_or (truth==1, truth==10), 2, labels)
        labels = np.where(np.logical_and(truth>=7, truth<= 9), 3, labels)
    elif n_classes == 9:
        labels = data['p_iffTruth']
        labels = np.where(labels== 9, 4, labels)
        labels = np.where(labels==10, 6, labels)
    else:
        print('\nCLASSIFIER:', n_classes, 'classes not supported -> exiting program\n')
        sys.exit()
    return labels

In [16]:
n_classes = 2
y = make_labels(xtrain_data, n_classes)
print(y.shape)

(19920,)


In [17]:
x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)
shuffle_idx = np.random.permutation(x.shape[0])
x = x[shuffle_idx]
y = y[shuffle_idx]

In [18]:
train_frac = 5/6
idx = int(train_frac * len(x))
x_train, x_test = x[:idx], x[idx:]
y_train, y_test = y[:idx], y[idx:]

In [19]:
## Hyperparameters
#num_epochs=20
num_epochs=100
#num_classes=10
nw=1
bs1=1000
bs2=1000
#bs=64
#wd=1e-5
lr=3e-4
#momentum=0.9

In [20]:
train_tensor_x = torch.stack([torch.Tensor(i) for i in x_train])
train_tensor_y = torch.stack([torch.zeros([1])+i for i in y_train])
train_tensor_y = train_tensor_y.type(torch.long).squeeze(1)
#train_tensor_ycpu = train_tensor_y.to('cpu')
#train_tensor_xcpu = train_tensor_x.to('cpu')

In [21]:
print('\033[1;33m tensor_x type: \033[1;m', train_tensor_x.type())
print('\033[1;35m tensor_x dtype: \033[1;m', train_tensor_x.dtype)
print('\033[1;36m tensor_x num of dims: \033[1;m', train_tensor_x.dim())
print('\033[1;34m tensor_x Shape:\033[1;m', train_tensor_x.shape)
print('\033[1;33m tensor_y type: \033[1;m', train_tensor_y.type())
print('\033[1;35m tensor_y dtype: \033[1;m', train_tensor_y.dtype)
print('\033[1;36m tensor_y num of dims: \033[1;m', train_tensor_y.dim())
print('\033[1;34m tensor_y Shape:\033[1;m', train_tensor_y.shape)

 tensor_x type:  torch.FloatTensor
 tensor_x dtype:  torch.float32
 tensor_x num of dims:  4
 tensor_x Shape: torch.Size([16600, 7, 7, 11])
 tensor_y type:  torch.LongTensor
 tensor_y dtype:  torch.int64
 tensor_y num of dims:  1
 tensor_y Shape: torch.Size([16600])


In [22]:
test_tensor_x = torch.stack([torch.Tensor(i) for i in x_test])
test_tensor_y = torch.stack([torch.zeros([1])+i for i in y_test])
test_tensor_y = test_tensor_y.type(torch.long).squeeze(1)
#test_tensor_ycpu = test_tensor_y.to('cpu')
#test_tensor_xcpu = test_tensor_x.to('cpu')

In [23]:
print('\033[1;33m tensor_x type: \033[1;m', test_tensor_x.type())
print('\033[1;35m tensor_x dtype: \033[1;m', test_tensor_x.dtype)
print('\033[1;36m tensor_x num of dims: \033[1;m', test_tensor_x.dim())
print('\033[1;34m tensor_x Shape:\033[1;m', test_tensor_x.shape)
print('\033[1;33m tensor_x type: \033[1;m', test_tensor_y.type())
print('\033[1;35m tensor_x dtype: \033[1;m', test_tensor_y.dtype)
print('\033[1;36m tensor_x num of dims: \033[1;m', test_tensor_y.dim())
print('\033[1;34m tensor_x Shape:\033[1;m', test_tensor_y.shape)

 tensor_x type:  torch.FloatTensor
 tensor_x dtype:  torch.float32
 tensor_x num of dims:  4
 tensor_x Shape: torch.Size([3320, 7, 7, 11])
 tensor_x type:  torch.LongTensor
 tensor_x dtype:  torch.int64
 tensor_x num of dims:  1
 tensor_x Shape: torch.Size([3320])


In [24]:
train_transformer = transforms.Compose([transforms.ToTensor()])
val_transformer = transforms.Compose([transforms.ToTensor()])

In [25]:
train_tensor_x.transform=train_transformer
test_tensor_x.transform=val_transformer

In [26]:
train_dataset = torch.utils.data.TensorDataset(train_tensor_x, train_tensor_y) 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs1, num_workers=nw, shuffle=True) 
valid_dataset = torch.utils.data.TensorDataset(test_tensor_x, test_tensor_y) 
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=bs2, num_workers=nw, shuffle=False) 

In [27]:
print(len(train_dataset))

16600


In [28]:
loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_tensor_x), num_workers=1)
data = next(iter(loader))
data[0].mean(), data[0].std()

(tensor(0.0002), tensor(1.0101))

In [29]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [30]:
device = get_default_device()
device

device(type='cuda')

In [31]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [32]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [33]:
train_loader = DeviceDataLoader(train_loader, device)
valid_loader = DeviceDataLoader(valid_loader, device)

In [34]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [35]:
print(len(train_loader))

17


In [36]:
print(len(valid_loader))

4


In [37]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [38]:
device = get_default_device()
device

device(type='cuda')

In [39]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [40]:
@torch.no_grad()
def evaluate(net, valid_loader):
    net.eval()
    outputs = [net.validation_step(batch) for batch in valid_loader]
    return net.validation_epoch_end(outputs)

def fit(epochs, lr, net, train_loader, valid_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(net.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        net.train()
        train_losses = []
        for batch in train_loader:
            loss = net.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation net
        result = evaluate(net, valid_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        net.epoch_end(epoch, result)
        history.append(result)
    return history

In [41]:
class CnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(7, 32, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            #nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(32, 32, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            #nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(32, 32, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            #nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(1), 
            nn.Linear(13*17*16, 100),
            nn.ReLU(),
            nn.Linear(100, 2))
        
    def forward(self, xb):
        return self.network(xb)

In [42]:
net = CnnModel()
net

CnnModel(
  (network): Sequential(
    (0): Conv2d(7, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Flatten()
    (13): Linear(in_features=3536, out_features=100, bias=True)
    (14): ReLU()
    (15): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [43]:
net = to_device(CnnModel(), device)

In [44]:
evaluate(net, valid_loader)

{'val_loss': 0.6759819388389587, 'val_acc': 0.6339687705039978}

In [45]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

import optim ######run the optim.py file


######################################
#ap('-m', help='mnistfc | mnistconv | allcnn', type=str, default='mnistconv')
#ap('-b',help='Batch size', type=int, default=128)
#ap('-B', help='Max epochs', type=int, default=100)
#ap('--lr', help='Learning rate', type=float, default=0.1)
#ap('--l2', help='L2', type=float, default=0.0)
#ap('-L', help='Langevin iterations', type=int, default=0)
#ap('--gamma', help='gamma', type=float, default=1e-4)
#ap('--scoping', help='scoping', type=float, default=1e-3)
#ap('--noise', help='SGLD noise', type=float, default=1e-4)
#ap('-g', help='GPU idx.', type=int, default=0)
#ap('-s', help='seed', type=int, default=42)

###optimizer parameters
#optimizer = optim.EntropySGD(model.parameters(),
        #config = dict(lr=opt['lr'], momentum=0.9, nesterov=True, weight_decay=opt['l2'],
        #L=opt['L'], eps=opt['noise'], g0=opt['gamma'], g1=opt['scoping']))

optimizer =  optim.EntropySGD(net.parameters(),
        config = dict(lr=0.0003, momentum=0.9, nesterov=True, weight_decay=1e-3,
        L=20, eps=1e-4, g0=0.03, g1=1e-3))


In [46]:
import matplotlib.pyplot as plt
import numpy as np
  
#from __future__ import print_function

exec('from __future__ import print_function')
import argparse, math, random
import torch as th
import torch.nn as nn
from torch.autograd import Variable
from timeit import default_timer as timer
import torch.backends.cudnn as cudnn

import models, loader, optim
import numpy as np
from utils import *

In [47]:
import torch_utils
from timeit import default_timer as timer



print(len(train_loader)/27)

def train(e):
    net.train()

    fs, top1 = AverageMeter(),  AverageMeter()
    ts = timer()

    bsz = 5000
    maxb = 10
    #maxb = int(math.ceil(len(train_loader)/bsz))
    #print(len(trainloader))
   
  

    for bi in range(maxb):
      #helper() is closure
        def helper():
            def feval():
                x,y = next(iter(train_loader))
                if torch.cuda.is_available():
                    x,y = x.cuda(), y.cuda()

                x, y = Variable(x), Variable(y.squeeze())
                bsz = x.size(0)

                optimizer.zero_grad()
                yh = net(x)
                f = criterion.forward(yh, y)
                f.backward()

                prec1, = accuracy(yh.data, y.data, topk=(1,))
                err = 100.-prec1.item()
                return (f.data.item(), err)
            return feval
      
        f, err = optimizer.step(helper(), net, criterion)

        fs.update(f, bsz)
        top1.update(err, bsz)

        if bi % 100 == 0 and bi != 0:
            print('[%2d][%4d/%4d] %2.4f %2.2f%%'%(e,bi,maxb, fs.avg, top1.avg))

    print('Train: [%2d] %2.4f %2.2f%% [%.2fs]'% (e, fs.avg, top1.avg, timer()-ts))
    print()

0.6296296296296297


In [48]:
def set_dropout(cache = None, p=0):
    if cache is None:
        cache = []
        for l in net.modules(): 
            if 'Dropout' in str(type(l)):
                cache.append(l.p)
                l.p = p
        return cache
    else:
        for l in net.modules():
            if 'Dropout' in str(type(l)):
                assert len(cache) > 0, 'cache is empty'
                l.p = cache.pop(0)

def dry_feed():
    cache = set_dropout()
    bsz = 5000
   
    maxb = int(math.ceil(len(train_loader)/bsz))
    for bi in range(maxb):
        x,y = next(iter(train_loader))
        if torch.cuda.is_available():
            x,y = x.cuda(), y.cuda()
        x,y =   Variable(x, volatile=True), \
                Variable(y.squeeze(), volatile=True)
        yh = net(x)
    set_dropout(cache)

def val(e, loader):
    dry_feed()
    net.eval()
    bsz = 128
    maxb = 10
    #maxb = int(math.ceil(len(train_loader)/bsz))

    fs, top1 = AverageMeter(), AverageMeter()
    for bi in range(maxb):
        x,y = next(iter(loader))
        bsz = x.size(0)

        if torch.cuda.is_available():
            x,y = x.cuda(), y.cuda()

        x,y =   Variable(x, ), \
                Variable(y.squeeze(), volatile=True)
        yh = net(x)

        f = criterion.forward(yh, y).item()
        prec1, = accuracy(yh.data, y.data, topk=(1,))
        err = 100-prec1.item()

        fs.update(f, bsz)
        top1.update(err, bsz)

    print('Test: [%2d] %2.4f %2.4f%%\n'%(e, fs.avg, top1.avg))
    print()


In [ ]:
for e in range(10):
    train(e)
    if e % 5 == 0:
        val(e, valid_loader)

Train: [ 0] 0.6757 36.63% [50.17s]

Test: [ 0] 0.6751 36.5000%


Train: [ 1] 0.6761 37.46% [44.35s]

Train: [ 2] 0.6745 37.14% [51.98s]

Train: [ 3] 0.6727 36.80% [46.08s]

Train: [ 4] 0.6713 36.61% [48.13s]

Train: [ 5] 0.6695 36.26% [46.28s]

Test: [ 5] 0.6694 36.5000%


Train: [ 6] 0.6698 36.85% [48.82s]

